### 1. Importación de Datos:
Utilice el conjunto de datos IMDB proporcionado por Keras. pero
esta vez, en lugar de utilizar sólo las 20.000 palabras más frecuentes, utilice las 50.000
palabras más frecuentes

In [20]:
# Importando las bibliotecas y funciones necesarias.
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [21]:
# Cargando el conjunto de datos IMDB con las 50,000 palabras más frecuentes.
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=50000)

Cargando los datos...


### 2. Pre-procesamiento:

• Secuencie y rellene las críticas para que todas tengan una longitud uniforme.

• De las críticas, extraiga características (features) adicionales, por ejemplo. la
longitud de la crítica, la proporción de palabras positivas/negativas y cualquier
otra que considere pueda ser útil.

In [22]:
# Realizando preprocesamiento en los datos: ajustando la longitud de las críticas a un tamaño uniforme.
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 500  # Longitud máxima de la secuencia
X_train = sequence.pad_sequences(X_train, maxlen = max_sequence_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_sequence_length)

In [23]:
# Obteniendo el índice de palabras del conjunto de datos IMDB.
word_to_id = imdb.get_word_index()

In [24]:
# Invertiendo el índice de palabras para obtener una correspondencia del ID de la palabra a la palabra real.
id_to_word = {value: key for key, value in word_to_id.items()}

In [25]:
# Función para calcular la proporción de palabras positivas a negativas en una crítica.
def calcular_ratio_positivo_negativo(sequence):
    palabras_positivas = ["good", "excellent", "wonderful"]  # IMDB está en inglés
    palabras_negativas = ["bad", "terrible", "horrible"]

    # Convertir secuencia a palabras
    words = [id_to_word.get(i, '') for i in sequence]

    # Contar palabras positivas y negativas en el texto
    num_palabras_positivas = sum(1 for palabra in words if palabra in palabras_positivas)
    num_palabras_negativas = sum(1 for palabra in words if palabra in palabras_negativas)

    # Calcular la proporción
    if num_palabras_negativas == 0:
        return num_palabras_positivas
    else:
        return num_palabras_positivas / num_palabras_negativas

In [26]:
# Extracción de características adicionales como la longitud de la crítica y la proporción de palabras positivas/negativas.
import numpy as np

# Longitud de la crítica
lengths_train = np.array([len(seq) for seq in X_train])
lengths_test = np.array([len(seq) for seq in X_test])

# Proporción de palabras positivas/negativas (esto debe calcularse con datos previamente etiquetados)
positive_words_ratio_train = np.array([calcular_ratio_positivo_negativo(seq) for seq in X_train])
positive_words_ratio_test = np.array([calcular_ratio_positivo_negativo(seq) for seq in X_test])

In [27]:
# Redimensionando las características extraídas para que tengan la forma adecuada.
lengths_train = lengths_train.reshape(-1, 1)
lengths_test = lengths_test.reshape(-1, 1)
positive_words_ratio_train = positive_words_ratio_train.reshape(-1, 1)
positive_words_ratio_test = positive_words_ratio_test.reshape(-1, 1)

In [28]:
# Combinando las características adicionales.
additional_features_train = np.hstack([lengths_train, positive_words_ratio_train])
additional_features_test = np.hstack([lengths_test, positive_words_ratio_test])

### 3. Modelo:

• Cree un modelo LSTM que acepte las características (features) adicionales junto
con la secuencia de palabras.

• Intente usar una arquitectura más compleja, incorporando más capas LSTM, capas
de Dropout para la regularización y tal vez alguna capa densamente conectada
después de la LSTM. (ver también la referencia al final de este documento)

In [30]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Entrada de secuencia
input_seq = Input(shape=(max_sequence_length,))
embedding = Embedding(50000, 128)(input_seq)

# Primera capa LSTM con dropout
lstm1 = LSTM(128, return_sequences=True, dropout=0.3)(embedding)

# Segunda capa LSTM con dropout y conexión residual
lstm2 = LSTM(64, return_sequences=True, dropout=0.2)(lstm1)
residual_connection = concatenate([lstm1, lstm2])  # Conexión residual
lstm2_res = LSTM(64, dropout=0.2)(residual_connection)

# Entrada de características adicionales
input_features = Input(shape=(2,))  # Aquí asumimos que tenemos 2 características adicionales

# Combinar ambas entradas
merged = concatenate([lstm2_res, input_features])

# Agregar más capas
dropout = Dropout(0.2)(merged)
dense1 = Dense(64, activation='relu')(dropout)
dense2 = Dense(32, activation='relu')(dense1)
output = Dense(1, activation='sigmoid')(dense2)

# Crear y compilar el modelo con un optimizador y tasa de aprendizaje ajustada
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Puedes experimentar con esta tasa
model = Model(inputs=[input_seq, input_features], outputs=output)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Implementación del Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

Epoch 1/15
391/391 - 102s - loss: 0.6550 - accuracy: 0.7465 - val_loss: 0.3562 - val_accuracy: 0.8455 - 102s/epoch - 260ms/step
Epoch 2/15
391/391 - 76s - loss: 0.2226 - accuracy: 0.9181 - val_loss: 0.3577 - val_accuracy: 0.8670 - 76s/epoch - 193ms/step
Epoch 3/15
391/391 - 54s - loss: 0.1427 - accuracy: 0.9510 - val_loss: 0.4699 - val_accuracy: 0.8610 - 54s/epoch - 137ms/step
Epoch 4/15
391/391 - 52s - loss: 0.1056 - accuracy: 0.9655 - val_loss: 0.4829 - val_accuracy: 0.8530 - 52s/epoch - 133ms/step


### 4. Entrenamiento y Evaluación: 
Entrene su modelo con el conjunto de datos de entrenamiento y evalúe su desempeño con el conjunto de datos de prueba.

In [ ]:
# Entrenar el modelo con callback de Early Stopping
model.fit([X_train, additional_features_train], y_train,
          batch_size=64,  # Aumentado el tamaño del lote para una convergencia más estable
          epochs=15,
          verbose=2,
          validation_data=([X_test, additional_features_test], y_test),
          callbacks=[early_stopping])

In [31]:
# Evaluando el modelo y mostrando resultados.
perdida, exactitud = model.evaluate([X_test, additional_features_test], y_test,
                                    batch_size = 32,
                                    verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)


782/782 - 15s - loss: 0.4829 - accuracy: 0.8530 - 15s/epoch - 20ms/step
Pérdida de la Prueba: 0.4828959107398987
Exactitud de la Prueba (Test accuracy): 0.8529599905014038


In [32]:
# Guardando el modelo entrenado.
model.save("Sentimiento.h5")

In [33]:
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3
# Retrieve the training sequences.
(X_train, _), _ = tf.keras.datasets.imdb.load_data(
    start_char=start_char, oov_char=oov_char, index_from=index_from
)
# Retrieve the word index file mapping words to indices
word_index = tf.keras.datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in X_train[0])

In [34]:
# Mostrando la secuencia decodificada.
decoded_sequence

"[START] this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh